In [3]:
# import os
# import sys
# sys.path.insert(0, "../src/")
# import re
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from handcrafted_features import DocBasedFeatureExtractor, Doc2VecChunkVectorizer, CorpusBasedFeatureExtractor
# from utils import get_doc_paths, read_labels
# 
# raw_docs_dir = "../data/raw_docs/"
# labels_dir = "../data/labels/"
# extracted_features_dir = "../data/extracted_features/"
# 
# lang = "eng"
# doc_paths = get_doc_paths(raw_docs_dir, lang)
# 
# sentences_per_chunk = 200
# d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
# d2vcv.fit_transform(doc_paths)


In [4]:
# all_chunk_based_features = []
# all_book_based_features = []
# all_average_sbert_sentence_embeddings = []
# all_doc2vec_chunk_embeddings = []
# for doc_path in tqdm(doc_paths):
#     fe = DocBasedFeatureExtractor(lang, doc_path, sentences_per_chunk)
#     chunk_based_features, book_based_features, average_sbert_sentence_embeddings, doc2vec_chunk_embeddings = fe.get_all_features()
#     all_chunk_based_features.extend(chunk_based_features)
#     all_book_based_features.append(book_based_features)
#     all_average_sbert_sentence_embeddings.append(average_sbert_sentence_embeddings)
#     all_doc2vec_chunk_embeddings.append(doc2vec_chunk_embeddings)


  2%|▏         | 13/599 [01:01<33:34,  3.44s/it]  /Users/arda/conda_root/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/arda/conda_root/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 599/599 [41:55<00:00,  4.20s/it]  


In [5]:
# cbfe = CorpusBasedFeatureExtractor(lang, doc_paths, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings)
# all_corpus_based_features = cbfe.get_all_features()


100%|██████████| 599/599 [00:52<00:00, 11.47it/s]
INFO:gensim.models.ldamodel:using symmetric alpha at 0.1
INFO:gensim.models.ldamodel:using symmetric eta at 0.1
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 10 topics, 2 passes over the supplied corpus of 599 documents, updating every 6000 documents, evaluating every ~599 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 3 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #599/599, outstanding queue size 1
DEBUG:gensim.models.ldamodel:updating topics
INFO:gensim.models.ldamodel:topic #6 (0.100): 0.011*"cecilia" + 0.007*"helen" + 0.004*"lucy" + 0.004*"jones" + 0.003*"sophia" + 0.003*"tho" + 0.003*"emily" + 0.002*"charlotte" + 0.002*"allan" + 0.002*"aurora"
INFO:gensim.models.ldamodel:topic #9 (0.100): 0.013*"laura" + 0.008*"molly" + 0.006

In [6]:
# import os
# import pandas as pd
# 
# book_df = pd.DataFrame(all_book_based_features)
# book_df = book_df.merge(all_corpus_based_features, on="book_name")
# book_and_averaged_chunk_df = book_df.merge(pd.DataFrame(all_chunk_based_features).groupby("book_name").mean().reset_index(drop=False), on="book_name")
# 
# chunk_df = pd.DataFrame(all_chunk_based_features)
# chunk_and_copied_book_df = chunk_df.merge(pd.DataFrame(all_book_based_features), on="book_name")
# chunk_and_copied_book_df = chunk_and_copied_book_df.merge(all_corpus_based_features, on="book_name")
# 
# os.makedirs(f"{extracted_features_dir}/{lang}", exist_ok=True)
# book_df.to_csv(f"{extracted_features_dir}/{lang}/book_df.csv", index=False)
# book_and_averaged_chunk_df.to_csv(f"{extracted_features_dir}/{lang}/book_and_averaged_chunk_df.csv", index=False)
# chunk_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_df.csv", index=False)
# chunk_and_copied_book_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_and_copied_book_df.csv", index=False)


In [1]:
import sys
sys.path.insert(0, "../src/")
import numpy as np
import pandas as pd

extracted_features_dir = "../data/extracted_features/"
labels_dir = "../data/labels/"
lang = "eng"

book_df = pd.read_csv(f"{extracted_features_dir}/{lang}/book_df.csv")
book_and_averaged_chunk_df = pd.read_csv(f"{extracted_features_dir}/{lang}/book_and_averaged_chunk_df.csv")
chunk_df = pd.read_csv(f"{extracted_features_dir}/{lang}/chunk_df.csv")
chunk_and_copied_book_df = pd.read_csv(f"{extracted_features_dir}/{lang}/chunk_and_copied_book_df.csv")


In [4]:
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from copy import deepcopy
from utils import read_labels
from sklearn.metrics import mean_squared_error, mean_absolute_error

labels = read_labels(labels_dir, lang)

class Experiment(object):
    def __init__(self, features, drop_columns_including, dimensionality_reduction, model, verbose):
        assert features in ["book", "chunk", "book_and_averaged_chunk", "chunk_and_copied_book"]
        assert isinstance(drop_columns_including, list)
        for i in drop_columns_including:
            assert isinstance(i, str)
        assert model in ["xgboost", "svr", "lasso"]
        assert (dimensionality_reduction in ["ss_pca_0_95", "k_best_f_reg_0_10", "k_best_mutual_info_0_10"]) or (dimensionality_reduction is None)
        self.features = features
        self.labels = labels
        self.drop_columns_including = drop_columns_including
        self.dimensionality_reduction = dimensionality_reduction
        self.model = model
        self.verbose = verbose

        if features == "book":
            self.df = deepcopy(book_df)
        elif features == "chunk":
            self.df = deepcopy(chunk_df)
        elif features == "chunk_and_copied_book":
            self.df = deepcopy(chunk_and_copied_book_df)
        elif features == "book_and_averaged_chunk":
            self.df = deepcopy(book_and_averaged_chunk_df)

        columns_before_drop = set(self.df.columns)
        self.df = self.df[[column for column in self.df.columns if not self._drop_column(column)]].reset_index(drop=True)
        columns_after_drop = set(self.df.columns)
        if self.verbose:
            print(f"Dropped {len(columns_before_drop - columns_after_drop)} columns.")
        self.df.loc[:, "y"] = self.df.book_name.apply(lambda x: self.labels[x]).tolist()

    def _drop_column(self, column):
        for string in self.drop_columns_including:
            if string in column:
                return True
        return False
    
    def _custom_pca(self, train_X):
        for i in range(5, train_X.shape[1], int((train_X.shape[1] - 5) / 10)):
            pca = PCA(n_components=i)
            new_train_X = pca.fit_transform(train_X)
            if pca.explained_variance_ratio_.sum() >= 0.95:
                break
        return new_train_X, pca

    def _select_features(self, train_X, train_y, validation_X):
        if self.dimensionality_reduction == "ss_pca_0_95":
            ss = StandardScaler()
            train_X = ss.fit_transform(train_X)
            validation_X = ss.transform(validation_X)
            train_X, pca = self._custom_pca(train_X)
            validation_X = pca.transform(validation_X)
        elif self.dimensionality_reduction == "k_best_f_reg_0_10":
            k_best = SelectKBest(f_regression, k=np.minimum(int(0.10 * train_X.shape[0]), train_X.shape[1]))
            train_X = k_best.fit_transform(train_X, train_y)
            validation_X = k_best.transform(validation_X)
        elif self.dimensionality_reduction == "k_best_mutual_info_0_10":
            k_best = SelectKBest(mutual_info_regression, k=np.minimum(int(0.10 * train_X.shape[0]), train_X.shape[1]))
            train_X = k_best.fit_transform(train_X, train_y)
            validation_X = k_best.transform(validation_X)
        elif self.dimensionality_reduction is None:
            pass
        return train_X, validation_X
    
    def _impute(self, train_X, validation_X):
        imputer = KNNImputer()
        train_X = imputer.fit_transform(train_X)
        validation_X = imputer.transform(validation_X)
        return train_X, validation_X
    
    def _get_model(self):
        # if any of these performs better than others, we can try to tune the hyperparameters
        # but I think for now it's more important to see which approach performs better
        # chunk based or doc based
        # use dimensionality reduction or not...
        if self.model == "xgboost":
            return XGBRegressor()
        elif self.model == "svr":
            return SVR()
        elif self.model == "lasso":
            return Lasso()
            
    def run(self):
        all_predictions = []
        all_labels = []
        train_mses = []
        train_maes = []
        validation_mses = []
        validation_maes = []

        df = self.df
        book_names = df['book_name'].unique()
        book_names_splitted = np.array_split(book_names, 10)
        for index, split in enumerate(book_names_splitted):
            train_df = df[~df["book_name"].isin(split)]
            validation_df = df[df["book_name"].isin(split)]
            train_X = train_df.drop(columns=["y", "book_name"]).values
            train_y = train_df["y"].values.ravel()
            validation_X = validation_df.drop(columns=["y", "book_name"]).values
            validation_y = validation_df["y"].values.ravel()
            train_X, validation_X = self._impute(train_X, validation_X)
            if self.verbose:
                print(f"train_X.shape before {self.dimensionality_reduction}: {train_X.shape}, validation_X.shape before {self.dimensionality_reduction}: {validation_X.shape}")
            train_X, validation_X = self._select_features(train_X, train_y, validation_X)
            if self.verbose:
                print(f"train_X.shape after {self.dimensionality_reduction}: {train_X.shape}, validation_X.shape after {self.dimensionality_reduction}: {validation_X.shape}")
            model = self._get_model()
            model.fit(train_X, train_y)
            
            train_books = deepcopy(train_df[["book_name", "y"]])
            train_books["yhat"] = model.predict(train_X)
            validation_books = deepcopy(validation_df[["book_name", "y"]])
            validation_books["yhat"] = model.predict(validation_X)
            
            train_books = train_books.groupby("book_name").mean()
            validation_books = validation_books.groupby("book_name").mean()
            
            train_y = train_books["y"].tolist()
            train_yhat = train_books["yhat"].tolist()
            validation_y = validation_books["y"].tolist()
            validation_yhat = validation_books["yhat"].tolist()
            
            all_labels.extend(validation_y)
            all_predictions.extend(validation_yhat)
            
            train_mse = mean_squared_error(train_y, train_yhat)
            train_mae = mean_absolute_error(train_y, train_yhat)
            validation_mse = mean_squared_error(validation_y, validation_yhat)
            validation_mae = mean_absolute_error(validation_y, validation_yhat)
            train_mses.append(train_mse)
            train_maes.append(train_mae)
            validation_mses.append(validation_mse)
            validation_maes.append(validation_mae)
            if self.verbose:
                print(f"Fold: {index+1}, TrainMSE: {np.round(train_mse, 3)}, TrainMAE: {np.round(train_mae, 3)}, ValMSE: {np.round(validation_mse, 3)}, ValMAE: {np.round(validation_mae, 3)}")
        all_labels = np.array(all_labels)
        all_predictions = np.array(all_predictions)

        mean_train_mse = np.mean(train_mses)
        mean_train_mae = np.mean(train_maes)
        mean_validation_mse = np.mean(validation_mses)
        mean_validation_mae = np.mean(validation_maes)
        
        if self.verbose:
            print("------")
            print(f"Mean scores, TrainMSE: {np.round(mean_train_mse, 3)}, TrainMAE: {np.round(mean_train_mae, 3)}, ValMSE: {np.round(mean_validation_mse, 3)}, ValMAE: {np.round(mean_validation_mae, 3)}")

            plt.figure(figsize=(18, 6))
            plt.scatter(all_labels, all_predictions)
            plt.xlabel("Ground Truths")
            plt.ylabel("Predictions")

            plt.show();
        return mean_train_mse, mean_train_mae, mean_validation_mse, mean_validation_mae


In [8]:
results = []

for model in ["xgboost", "lasso", "svr"]:
    for features in ["book", "chunk", "book_and_averaged_chunk", "chunk_and_copied_book"]:
        for drop_columns_including in [["doc2vec_chunk_embedding"], ["average_sentence_embedding"], ["average_sentence_embedding", "doc2vec_chunk_embedding"], []]:
            for dimensionality_reduction in ["k_best_f_reg_0_10", "ss_pca_0_95", "k_best_mutual_info_0_10", None]:
                try:
                    experiment = Experiment(
                        features=features,
                        drop_columns_including=drop_columns_including,
                        dimensionality_reduction=dimensionality_reduction,
                        model=model,
                        verbose=False
                    )
                    train_mse, train_mae, validation_mse, validation_mae = experiment.run()
                    results.append((model, features, drop_columns_including, dimensionality_reduction, train_mse, train_mae, validation_mse, validation_mae))
                    print(model, features, drop_columns_including, dimensionality_reduction, train_mse, train_mae, validation_mse, validation_mae)
                except Exception as e:
                    print(f"Error in {model}, {features}, {drop_columns_including}, {dimensionality_reduction}")
                    print(e)


xgboost book ['doc2vec_chunk_embedding'] k_best_f_reg_0_10 0.0001331986638318581 0.00781272138513123 497.4080857867046 17.596386708955617
xgboost book ['doc2vec_chunk_embedding'] ss_pca_0_95 1.9417901707297977e-07 0.00030323384584537473 548.6991066855837 18.76817986940835
xgboost book ['doc2vec_chunk_embedding'] k_best_mutual_info_0_10 0.0001803956307178797 0.00867681814712686 522.5112248261811 18.108052991031606
xgboost book ['doc2vec_chunk_embedding'] None 2.3294693816108048e-07 0.0003448382162276392 526.6013000346694 18.112050449598218
xgboost book ['average_sentence_embedding'] k_best_f_reg_0_10 0.0001331986638318581 0.00781272138513123 497.4080857867046 17.596386708955617
xgboost book ['average_sentence_embedding'] ss_pca_0_95 1.8744597843288232e-07 0.0002992014086068399 555.2859491844622 19.1250903861175
xgboost book ['average_sentence_embedding'] k_best_mutual_info_0_10 0.00015988937505826816 0.008340205812670382 519.8295855023355 17.982910718196113
xgboost book ['average_senten

xgboost chunk_and_copied_book ['average_sentence_embedding', 'doc2vec_chunk_embedding'] ss_pca_0_95 0.058006408082086215 0.1546833677470367 530.7842588294244 18.5082530411777
xgboost chunk_and_copied_book ['average_sentence_embedding', 'doc2vec_chunk_embedding'] k_best_mutual_info_0_10 4.2247544900260794e-05 0.0037746997660272367 533.9720578643366 18.22458935880265
xgboost chunk_and_copied_book ['average_sentence_embedding', 'doc2vec_chunk_embedding'] None 4.2247544900260794e-05 0.0037746997660272367 533.9720578643366 18.22458935880265
xgboost chunk_and_copied_book [] k_best_f_reg_0_10 9.940350901856796e-05 0.005869913853120751 528.7035362252121 18.072510760377778
xgboost chunk_and_copied_book [] ss_pca_0_95 1.341308862411226 0.8556929400277353 489.1217990042008 18.08186397315982
xgboost chunk_and_copied_book [] k_best_mutual_info_0_10 9.940350901856796e-05 0.005869913853120751 528.7035362252121 18.072510760377778
xgboost chunk_and_copied_book [] None 9.940350901856796e-05 0.0058699138

/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2233083.3903988823, tolerance: 936.6783161665354
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3617004.5691486043, tolerance: 909.2665373937298
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3874698.2968934886, tolerance: 895.5732188672007
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinat

lasso chunk ['doc2vec_chunk_embedding'] k_best_f_reg_0_10 616.8146790970343 20.337788681452786 624.7833594620961 20.444842754414584
lasso chunk ['doc2vec_chunk_embedding'] ss_pca_0_95 524.6255215574992 18.715786392848347 589.1074095231891 19.782115858963046


/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2233083.3903988823, tolerance: 936.6783161665354
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3617004.5691486043, tolerance: 909.2665373937298
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3874698.2968934886, tolerance: 895.5732188672007
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinat

...

/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2401396.7024007244, tolerance: 936.6783161665354
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2920298.8446179926, tolerance: 909.2665373937298
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2720754.328847845, tolerance: 895.5732188672007
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate

lasso chunk_and_copied_book [] k_best_mutual_info_0_10 594.488589513087 19.08798387689692 745.8868453289335 21.526981146052258


/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2401396.7024007244, tolerance: 936.6783161665354
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2920298.8446179926, tolerance: 909.2665373937298
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2720754.328847845, tolerance: 895.5732188672007
  model = cd_fast.enet_coordinate_descent(
/Users/arda/conda_root/lib/python3.8/site-packages/sklearn/linear_model/_coordinate

lasso chunk_and_copied_book [] None 594.488589513087 19.08798387689692 745.8868453289335 21.526981146052258
svr book ['doc2vec_chunk_embedding'] k_best_f_reg_0_10 563.5022073446632 18.89424522080399 593.338704535949 19.55447141250492
svr book ['doc2vec_chunk_embedding'] ss_pca_0_95 546.2729255987458 18.498132310604895 577.1901673523931 19.332569443793535
svr book ['doc2vec_chunk_embedding'] k_best_mutual_info_0_10 573.3596844270687 19.084110075595646 593.8664490575935 19.650829097527463
svr book ['doc2vec_chunk_embedding'] None 560.1215631133236 18.73612029352963 589.5175152578726 19.52248256075103
svr book ['average_sentence_embedding'] k_best_f_reg_0_10 563.5022073446632 18.89424522080399 593.338704535949 19.55447141250492
svr book ['average_sentence_embedding'] ss_pca_0_95 546.2849203123006 18.498283913463702 577.0975669966313 19.330405402047457
svr book ['average_sentence_embedding'] k_best_mutual_info_0_10 576.7431003127623 19.140681343945666 599.8615112537261 19.764963725413644
s

svr chunk_and_copied_book [] k_best_f_reg_0_10 609.7590130368773 20.321421199371617 625.3401137269715 20.64032518834744
svr chunk_and_copied_book [] ss_pca_0_95 312.1932703399802 13.001115805297129 465.07342999716104 17.265608861522523
svr chunk_and_copied_book [] k_best_mutual_info_0_10 609.7590130368773 20.321421199371617 625.3401137269715 20.64032518834744
svr chunk_and_copied_book [] None 609.7590130368773 20.321421199371617 625.3401137269715 20.64032518834744


# Validation MAE medians

In [18]:
results_df = pd.DataFrame(results, columns=["model", "features", "drop_columns_including", "dimensionality_reduction", "train_mse", "train_mae", "validation_mse", "validation_mae"])
results_df["drop_columns_including"] = results_df["drop_columns_including"].apply(str)


In [19]:
results_df.groupby("model").agg({"validation_mae": "median"})

,validation_mae
model,
lasso,19.765545
svr,19.824300
xgboost,18.102226


In [20]:
results_df.groupby("features").agg({"validation_mae": "median"})

,validation_mae
features,
book,19.241809
book_and_averaged_chunk,19.156807
chunk,19.690169
chunk_and_copied_book,20.178956


In [21]:
results_df.groupby("drop_columns_including").agg({"validation_mae": "median"})

,validation_mae
drop_columns_including,
"['average_sentence_embedding', 'doc2vec_chunk_embedding']",19.690169
['average_sentence_embedding'],19.145792
['doc2vec_chunk_embedding'],19.462405
[],19.153238


In [22]:
results_df.groupby("dimensionality_reduction").agg({"validation_mae": "median"})

,validation_mae
dimensionality_reduction,
k_best_f_reg_0_10,19.584980
k_best_mutual_info_0_10,19.836356
ss_pca_0_95,18.596719


# Best result

In [23]:
results_df[results_df.validation_mae == results_df.validation_mae.min()]

,model,features,drop_columns_including,dimensionality_reduction,train_mse,train_mae,validation_mse,validation_mae
39,xgboost,book_and_averaged_chunk,['average_sentence_embedding'],None,2.071329e-07,0.000327,468.173575,16.857061


In [24]:
results_df.to_csv("../data/results/model_features_drop_columns_including_dimensionality_reduction_eng.csv", index=False)
